# 第三章：分配角色（角色提示工程）

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re
import openai
import os

# 优先从环境变量读取API配置，如果没有则从IPython存储中检索
API_KEY = os.getenv('OPENAI_API_KEY')
BASE_URL = os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1')
MODEL_NAME = os.getenv('OPENAI_MODEL_NAME', 'gpt-4o')

# 如果环境变量中没有API_KEY，尝试从IPython存储中获取
if not API_KEY:
    try:
        %store -r API_KEY
        %store -r MODEL_NAME
    except:
        pass

# 如果没有设置MODEL_NAME，使用默认值
if 'MODEL_NAME' not in locals() or not MODEL_NAME:
    MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

# 创建OpenAI客户端实例
# 这是与GPT AI模型交互的核心接口
client = openai.OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

def get_completion(prompt: str, system_prompt=""):
    """
    获取GPT的完成响应
    
    参数:
        prompt (str): 用户提示内容
        system_prompt (str): 系统提示（可选），用于设定AI的角色和行为准则
    
    返回:
        str: GPT的响应文本
    """
    # 构建消息列表
    messages = []
    
    # 如果有系统提示，添加系统消息
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    # 添加用户消息
    messages.append({"role": "user", "content": prompt})
    
    # 调用OpenAI API
    response = client.chat.completions.create(
        model=MODEL_NAME,              # 使用的模型名称 (gpt-4o 或 deepseek-r1)
        messages=messages,             # 消息列表
        max_tokens=2000,              # 最大生成的token数量
        temperature=0.0               # 温度参数，0表示更确定性的回答
    )
    return response.choices[0].message.content

---

## 课程

延续GPT除了您所说的内容之外没有任何上下文这一主题，有时**提示GPT扮演特定角色（包括所有必要的上下文）**是很重要的。这也被称为角色提示工程。角色上下文的细节越丰富越好。

**为GPT分配角色可以显著改善GPT在各个领域的表现**，从创意写作到编程再到数据分析总结。这就像人类专家被告知"像一个资深专家一样思考"时会表现得更专业一样。角色提示还可以精确调整GPT响应的风格、语调和专业程度。

**注意：** 角色提示既可以在系统提示中进行设定，也可以作为用户消息轮次的一部分进行指定。

### 示例

在下面的示例中，我们看到如果没有角色提示，当被要求对滑板运动给出一句话的观点时，GPT提供了一个**直接且无风格化的标准答案**。

然而，当我们让GPT扮演猫的角色时，GPT的观点发生了根本性变化，因此**GPT的响应语调、表达风格、内容观点都完全适应了新的角色设定**。

**注意：** 您可以使用的一个进阶技巧是**为GPT提供其目标受众的具体上下文**。在下面，我们可以调整提示来告诉GPT它应该与谁对话。"您是一只猫"与"您是一只对着一群滑板爱好者说话的猫"会产生截然不同的响应效果。

这是在系统提示中没有角色提示的基础提示：

In [ ]:
# 没有角色设定的基础提示
# 这是一个标准的问题，没有给AI设定任何特殊角色或身份
PROMPT = "In one sentence, what do you think about skateboarding?"

# 打印GPT的响应（没有角色设定时的标准回答）
print(get_completion(PROMPT))

这是相同的用户问题，但使用了角色提示。

In [ ]:
# 设置系统提示，让GPT扮演猫的角色
# 这个角色设定会显著改变AI的回答风格和内容
SYSTEM_PROMPT = "You are a cat."

# 相同的用户提示
PROMPT = "In one sentence, what do you think about skateboarding?"

# 打印GPT的响应（注意角色设定后回答风格的变化）
print(get_completion(PROMPT, SYSTEM_PROMPT))

您可以使用角色提示作为一种方式来让GPT模仿某些写作风格、以某种声音说话，或引导其答案的复杂性。**角色提示还可以使GPT更好地执行数学或逻辑任务。**

例如，在下面的示例中，有一个明确的正确答案，即是的。但是，GPT回答错了，认为它缺乏信息，实际上它并不缺乏：

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print GPT's response
print(get_completion(PROMPT))

现在，如果我们**让GPT扮演逻辑机器人的角色**会怎样？这将如何改变GPT的答案？

事实证明，通过这种新的角色分配，GPT得到了正确答案。（尽管值得注意的是，并非出于所有正确的原因）

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print GPT's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**注意：** 您在本课程中将学到的是，有**许多提示工程技术可以用来得出类似的结果**。您使用哪些技术取决于您和您的偏好！我们鼓励您**尝试找到自己的提示工程风格**。

如果您想在不更改上述任何内容的情况下实验课程提示，请滚动到课程notebook的最底部访问[**示例演练场**](#example-playground)。

---

## 练习
- [练习 3.1 - 数学纠错](#exercise-31---math-correction)

### 练习 3.1 - 数学纠错
在某些情况下，**GPT在数学方面可能会有困难**，即使是简单的数学。下面，GPT错误地评估数学问题为正确解决，尽管第二步中有明显的算术错误。请注意，GPT在逐步检查时实际上发现了错误，但没有得出整体解决方案是错误的结论。

修改`PROMPT`和/或`SYSTEM_PROMPT`，使GPT将解决方案评为`错误`解决，而不是正确解决。 


In [ ]:
# 系统提示 - 如果您不想使用系统提示，可以将此变量设置为空字符串
# 提示：尝试设置一个能够准确识别数学错误的角色
SYSTEM_PROMPT = "You are a mathematics teacher who is very careful about checking calculations and identifying errors."

# 用户提示 - 包含一个明显的数学错误的问题
# 正确答案：2x = 12（而不是6），所以 x = 6（而不是3）
PROMPT = """Is this equation solved correctly below?

2x - 3 = 9
2x = 6
x = 3"""

# 获取GPT的响应
response = get_completion(PROMPT, SYSTEM_PROMPT)

# 评分练习正确性的函数
def grade_exercise(text):
    """检查文本是否表明解决方案是错误的"""
    if "incorrect" in text or "not correct" in text.lower():
        return True
    else:
        return False

# 打印GPT的响应和相应的分数
print(response)
print("\n--------------------------- 评分 ---------------------------")
print("本练习已正确解决:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
# 提示：尝试设定一个严格的数学检查者角色，比如"You are a strict mathematics teacher who carefully checks every step of calculations."
print("提示：尝试设定一个严格的数学检查者角色，让GPT仔细检查每一步计算。")

### 恭喜！

如果您已经解决了到此为止的所有练习，您就可以进入下一章了。愉快的提示工程！

---

## 示例演练场

这是一个让您自由实验本课程中显示的提示示例的区域，您可以调整提示来看看它如何影响GPT的响应。

In [ ]:
# 提示
PROMPT = "In one sentence, what do you think about skateboarding?"

# 打印GPT的响应
print(get_completion(PROMPT))

In [ ]:
# 系统提示
SYSTEM_PROMPT = "You are a cat."

# 用户提示
PROMPT = "In one sentence, what do you think about skateboarding?"

# 打印GPT的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print GPT's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print GPT's response
print(get_completion(PROMPT, SYSTEM_PROMPT))